In [1]:
import rpa as r
import os
from datetime import datetime
import calendar

In [2]:
class Streak():
    def __init__(self):
        self.r = r

        self.r.init()

        # go to streak
        self.r.url('https://streak.tech')

        # set a maximum 5-minute timeout for user to login
        self.r.timeout(300)

        # use exist() function with XPath to check if logged in
        if not self.r.exist('//*[@class="jss423"]'):
            self.r.dom(
                'alert("Quitting as login is not detected after 5 min. Bye!")')

        print('Logged in to streak.')

        # change back to default of 10 seconds to quit fast on error
        self.r.timeout(5)

    def __del__(self):
        self.r.close()

    def change_month(self, month):
        _sel_backtest_parameter_button = '//*[@id="tour_bt_6"]/div/div/div[6]/button'
        _sel_backtest_parameter_stop_date_button = '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[1]/div/input'
        _scroll_container = 'window.document.getElementById("tour_bt_6").getElementsByTagName("div")[0].getElementsByTagName("div")[0]'

        print(f'Changing month to {month}.')

        screen_month = self.r.read(
            '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[2]/div/table/thead/tr[1]/th[2]')
        screen_month_date = datetime.strptime(screen_month, "%B %Y")
        month_date = datetime.strptime(month, "%B %Y")

        change_month = (month_date.year - screen_month_date.year) * \
            12 + month_date.month - screen_month_date.month

        print(f'{change_month} month(s) to change.')

        # Scroll down
        self.r.wait(1)
        self.r.dom(f'{_scroll_container}.scrollBy(0, 1000)')

        # Open Backtest Parameters
        self.r.wait(1)
        self.r.click(_sel_backtest_parameter_button)

        self.r.wait(1)
        self.r.dom(f'{_scroll_container}.scrollBy(0, 1000)')

        # Backtesting stop date
        self.r.wait(1)
        self.r.click(_sel_backtest_parameter_stop_date_button)

        # Change month
        self.r.wait(1)
        if change_month > 0:
            for i in range(abs(change_month)):
                # Go right
                self.r.click(
                    '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[2]/div/table/thead/tr[1]/th[3]')
        else:
            for i in range(abs(change_month)):
                # Go left
                self.r.click(
                    '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[2]/div/table/thead/tr[1]/th[1]')

        # Calculate calendar col
        start_month_day, end_month_day = calendar.monthrange(
            month_date.year, month_date.month)
        col = datetime(month_date.year, month_date.month,
                       end_month_day).weekday() + 2
        col = (col - 7) if col > 7 else col
        col = col

        # Calculate calendar row
        row = 5

        # Select last date by row and col
        print(f'Selecting row {row} column {col} from calendar.')
        self.r.wait(1)
        self.r.dom(
            f'window.document.getElementsByTagName("tbody")[3].children[{row - 1}].children[{col - 1}].click()')

        # Check if row needs to be re-selected
        end_day = int(self.r.read(
            '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[1]/div/input').split(' ')[0])
        if ((end_day + 7) <= end_month_day):
            row = 6
            print(f'Re-selecting row {row} column {col} from calendar.')
            self.r.wait(1)
            self.r.dom(
                f'window.document.getElementsByTagName("tbody")[3].children[{row - 1}].children[{col - 1}].click()')

        # Close Backtest Parameters
        self.r.wait(1)
        self.r.click(_sel_backtest_parameter_button)

        # Scroll up
        self.r.wait(1)
        self.r.dom(f'{_scroll_container}.scrollBy(0, -1000)')

    def change_exit(self, stop_loss=None, target_profit=None):
        _sel_backtest_parameter_button = '//*[@id="tour_bt_6"]/div/div/div[5]/button'
        _sel_backtest_parameter_stop_loss = '//*[@id="tour_bt_6"]/div/div/div[5]/div/div/div[2]/div/div/div/input'
        _sel_backtest_parameter_target_profit = '//*[@id="tour_bt_6"]/div/div/div[5]/div/div/div[3]/div/div/div/input'
        _scroll_container = 'window.document.getElementById("tour_bt_6").getElementsByTagName("div")[0].getElementsByTagName("div")[0]'

        print(
            f'Changing stop loss to {stop_loss} and target profit to {target_profit}.')

        if(stop_loss is None and target_profit is None):
            print('Nothing to change. Exiting.')
            return

        # Scroll down
        self.r.dom(f'{_scroll_container}.scrollBy(0, 1000)')

        # Open Exit Parameters
        self.r.wait(1)
        self.r.click(_sel_backtest_parameter_button)

        self.r.dom(f'{_scroll_container}.scrollBy(0, 1000)')

        # Change stop loss
        if stop_loss:
            self.r.wait(1)
            self.r.type(_sel_backtest_parameter_stop_loss,
                        f'[clear]{stop_loss}')
            print(f'Changed stop loss to {stop_loss}.')

        # Change target profit
        if target_profit:
            self.r.wait(1)
            self.r.type(_sel_backtest_parameter_target_profit,
                        f'[clear]{target_profit}')
            print(f'Changed target profit to {target_profit}.')

        # Close Exit Parameters
        self.r.wait(1)
        self.r.click(_sel_backtest_parameter_button)

        # Scroll up
        self.r.wait(1)
        self.r.dom(f'{_scroll_container}.scrollBy(0, -1000)')
        self.r.wait(1)

    def algo(self, uuid):
        print(f'Switched to algo:{uuid}')

        # go to strategy page
        self.r.url(f'https://www.streak.tech/backtests?algo_uuid={uuid}')

    def backtest(self, month, output=None, retry_count=3, backtest_wait=20):
        # Get Month
        screen_month = self.r.read(
            '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[2]/div/table/thead/tr[1]/th[2]')
        print(f'Current month on screen:{screen_month}')

        self.r.wait(1)
        self.change_month(month)
        self.r.wait(1)

        # Validate month changed
        screen_month = self.r.read(
            '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[2]/div/table/thead/tr[1]/th[2]')
        if(screen_month != month):
            print('Failed to change month. Exiting.')
            return

        # Run Backtest
        start_date = self.r.read(
            '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[2]/div[1]/div/input')
        end_date = self.r.read(
            '//*[@id="tour_bt_6"]/div/div/div[6]/div/div/div[3]/div[1]/div/input')
        self.r.click('(//*[@id="tour_bt_6"]/div/div/div[8]/button)')
        print(f'Started backtest for {start_date} to {end_date}.')

        # Download prep
        file_id = self.r.read('//*[@id="screen_header"]/div/div/p[2]')
        if output:
            filename = f'data/{month}_{output}.csv'
        else:
            filename = f'data/{month}.csv'
        try:
            os.remove(filename)
        except OSError:
            pass

        # Try download else it is streak error
        for i in range(retry_count):
            print(f'Trying download count {i + 1}.')

            # Let backtest run
            self.r.wait(backtest_wait)

            # click menu
            self.r.hover(
                '//*[@id="authScreeens"]/div[2]/div/div/div/div[3]/button')
            self.r.wait(1)
            self.r.click(
                '//*[@id="authScreeens"]/div[2]/div/div/div/div[3]/button')

            # download all transactions
            self.r.click('//*[@id="main"]/div[4]/div/div[4]/button')
            self.r.wait(2)

            try:
                os.rename(f'{file_id}_transaction_details.csv', filename)
                print(f'Saved file to {filename}')
                self.r.wait(1)
                return
            except FileNotFoundError:
                pass
        print(f'Download failed for {filename}. Exiting.')
    
    def backtest_year(self, year, output):
        months = calendar.month_name[1:12]
        [self.backtest(month=f'{month} {year}', output=output) for month in months]


In [3]:
uuid = 'NGZlOGNkZjEtNTRkMC00ZmZmLTk1ZmMtZjBkMDk1NmYwNTk4'
months = calendar.month_name[1:12]

In [4]:
streak = Streak()
streak.algo(uuid=uuid)

Logged in to streak.
Switched to algo:NGZlOGNkZjEtNTRkMC00ZmZmLTk1ZmMtZjBkMDk1NmYwNTk4


In [5]:
# Stop loss 5, Target profit 200
streak.change_exit(stop_loss=5, target_profit=200)
streak.backtest_year(year=2021, output='sl5tp200')

Changing stop loss to 5 and target profit to 200.
Changed stop loss to 5.
Changed target profit to 200.
Current month on screen:January 2021
Changing month to January 2021.
0 month(s) to change.
Selecting row 5 column 1 from calendar.
Re-selecting row 6 column 1 from calendar.
Started backtest for 02 Jan 2021 to 31 Jan 2021.
Trying download count 1.
Saved file to data/January 2021_sl5tp200.csv
Current month on screen:January 2021
Changing month to February 2021.
1 month(s) to change.
Selecting row 5 column 1 from calendar.
Started backtest for 30 Jan 2021 to 28 Feb 2021.
Trying download count 1.
Saved file to data/February 2021_sl5tp200.csv
Current month on screen:February 2021
Changing month to March 2021.
1 month(s) to change.
Selecting row 5 column 4 from calendar.
Started backtest for 02 Mar 2021 to 31 Mar 2021.
Trying download count 1.
Trying download count 2.
Trying download count 3.
Saved file to data/March 2021_sl5tp200.csv
Current month on screen:March 2021
Changing month to A

In [ ]:
# Stop loss 10, Target profit 100
streak.change_exit(stop_loss=10, target_profit=100)
streak.backtest_year(year=2021, output='sl10tp100')

In [ ]:
del streak